# Israel Natural Gas Model

In [1]:
# load required packages 
import itertools
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from ixmp import ixPlatform

In [2]:
# launch the IX modeling platform using the local default database                                                                                                                       
mp = ixPlatform(dbtype='HSQLDB') # use dbtype='HSQLDB' for a local database

In [3]:
model = "Israeli Natural Gas Model"
scen = "baseline"
annot = "developing a stylized energy system model for illustration and testing" 

ds = mp.datastructure(model, scen, version='new', annotation=annot, scheme="MESSAGE")

# Time and Spatial 

In [4]:
horizon = [2010, 2020, 2030]
firstyear = horizon[0]

In [5]:
ds.add_set("year", horizon)
ds.add_set("cat_year", ["firstmodelyear", firstyear])

In [6]:
country = "Israel"
field = ["Leviatan", "KT", "Tamar"]
ds.add_set("node", country)
ds.add_set("node", field)

ds.add_set("lvl_spatial", ["country", "field"])
ds.add_set("map_spatial_hierarchy", ["country", country, "World"])
for n in field:
    ds.add_set("map_spatial_hierarchy", ["field", n, "World"])
ds.add_set("mode", "standard")

In [7]:
ds.add_set("commodity", ["gas"])
ds.add_set("level", ["resource", "primary", "secondary"])

# Economic Parameters

In [8]:
rate = [0.05] * len(horizon)
unit = ['%'] * len(horizon)
ds.add_par("interestrate", key=horizon, val=rate, unit=unit)

# Technologies

In [9]:
techs = [
    "gas_extraction",
    'pipe',
    'LNG_import',
    'transmission',
    'gas_export',
]

ds.add_set("technology", techs)

In [10]:
demand = [11, 13, 15]
gas_demand = pd.DataFrame({
        'node': "Israel",
        'commodity': 'gas',
        'level': 'secondary',
        'year': horizon,
        'time': 'year',
        'value': demand,
        'unit': 'GWa',
    })
ds.add_par("demand", gas_demand)

# Engineering Parameters

In [11]:
year_pairs = [(y_v, y_a) for y_v, y_a in itertools.product(horizon, horizon) if y_v <= y_a]
vintage_years, act_years = zip(*year_pairs)

In [12]:
for f in field:
    base_inout = {
        'node_loc': f,
        'year_vtg': vintage_years,
        'year_act': act_years,
        'mode': 'standard',
        'commodity': 'gas',
        'time': 'year', 
    }

    inp = pd.DataFrame(dict(
        technology = 'gas_extraction',
        level = 'resource',
        node_origin = f,
        time_origin = 'year',
        value = 1.05, #amount of extraction
        unit = '-',
        **base_inout
        ))

    ds.add_par("input", inp) 

    outp = pd.DataFrame(dict(
        technology = 'gas_extraction',
        level = 'primary',
        node_dest = f,
        time_dest = 'year', 
        value = 1.,
        unit = '-',
        **base_inout
        ))

    ds.add_par("output", outp) 

    


### Technoeconomic Parameters

In [13]:
def make_df(base, **kwargs):
    base.update(kwargs)
    return pd.DataFrame(base)

In [14]:
base_inv_cost = {
    'technology': "gas_extraction",
    'year_vtg': horizon,
    'unit': 'USD/GWa',
}

# in $ / kW
field_costs = {
    'Tamar': 100,
    'Leviatan':  200,
    'KT':  300,
}

for f, val in field_costs.items():
    df = make_df(base_inv_cost, node_loc=f, value=val)
    ds.add_par('inv_cost', df)
    
base_fix_cost = {
    'technology': "gas_extraction",
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}

# in $ / kW
field_costs = {
    'Tamar': 20,
    'Leviatan':  10,
    'KT':  15,
}

for f, val in field_costs.items():
    df = make_df(base_fix_cost, node_loc=f, value=val)
    ds.add_par('fix_cost', df)
    
base_var_cost = {
    'technology': "gas_extraction",
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

# in $ / GWa
field_costs = {
    'Tamar': 20,
    'Leviatan':  10,
    'KT':  15,
}

for f, val in field_costs.items():
    df = make_df(base_var_cost, node_loc=f, value=val)
    ds.add_par('var_cost', df)

In [15]:
base_technical_lifetime = {
    'technology': "gas_extraction",
    'node_loc': field,
    'year_vtg': horizon,
    'unit': 'y',
}

df = make_df(base_technical_lifetime, value=30)
ds.add_par('technical_lifetime', df)

In [16]:
ds.add_set('grade', 'a')

#BCM
base_vol = {
    'commodity': 'gas',
    'grade': 'a'
}

field_vol = {
    'Tamar': 350,
    'Leviatan': 500,
    'KT': 50,
}

for f, val in field_vol.items():
    print(f, val, base_vol)
    df = make_df(base_vol, node=f, value=val)
    print(df)
    ds.add_par('resource_volume', df)

ds.add_set('level_resource', 'resourc')

demand = [11, 13, 15]
gas_demand = pd.DataFrame({
        'node': "Israel",
        'commodity': 'gas',
        'level': 'secondary',
        'year': horizon,
        'time': 'year',
        'value': demand,
        'unit': 'GWa',
    })


Tamar 350 {'commodity': 'gas', 'grade': 'a'}


ValueError: If using all scalar values, you must pass an index